<a href="https://colab.research.google.com/github/41371108H/114-1repo/blob/main/recorder_with_greetings_csv_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [38]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1X0uCEvhBCrJ-M8_nWAnGi0bcacp_Nzt5-V_z5rK8rPc/edit?usp=sharing')


In [39]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('工作表1').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()


,日期,品項,時間,金額,問候語


In [40]:
import datetime
import random

def record_data():
    """
    讓使用者輸入日期、品項、時間和金額，並將資料寫入檔案。
    """
    filename = "log.txt"
    greetings = [
        "別花太多錢喔！",
        "祝您有愉快的一天！",
        "今天也要努力賺錢！",
        "繼續保持好習慣！",
        "每一筆花費都值得！"
    ]

    print("歡迎使用紀錄程式。")
    print("請按照提示輸入，輸入 'q' 退出。")
    print("-" * 20)

    while True:
        # 獲取日期
        date_input = input("請輸入日期 (例如: 2025-09-11 或 'today'): ")
        if date_input.lower() == 'q':
            break

        if date_input.lower() == 'today':
            date_to_record = datetime.date.today().strftime('%Y-%m-%d')
        else:
            try:
                datetime.datetime.strptime(date_input, '%Y-%m-%d')
                date_to_record = date_input
            except ValueError:
                print("日期格式不正確，請使用 YYYY-MM-DD 格式。")
                print("-" * 20)
                continue

        # 獲取品項
        item_input = input("請輸入品項: ")
        if item_input.lower() == 'q':
            break

        # 獲取時間
        time_input = input("請輸入時間 (例如: 14:30 或 'now'): ")
        if time_input.lower() == 'q':
            break

        if time_input.lower() == 'now':
            time_to_record = datetime.datetime.now().strftime('%H:%M')
        else:
            try:
                datetime.datetime.strptime(time_input, '%H:%M')
                time_to_record = time_input
            except ValueError:
                print("時間格式不正確，請使用 HH:MM 格式。")
                print("-" * 20)
                continue

        # 獲取金額
        amount_input = input("請輸入金額: ")
        if amount_input.lower() == 'q':
            break

        try:
            amount_to_record = float(amount_input)
        except ValueError:
            print("金額格式不正確，請輸入數字。")
            print("-" * 20)
            continue

        # 建立要寫入的字串 (不含標籤)
        log_entry_to_file = f"{date_to_record}, {item_input}, {time_to_record}, {amount_to_record}\n"

        # 建立要顯示在螢幕上的字串 (包含標籤)
        display_message = (
            f"成功新增紀錄: "
            f"日期: {date_to_record}, "
            f"品項: {item_input}, "
            f"時間: {time_to_record}, "
            f"金額: {amount_to_record}"
        )

        # 選擇一則隨機問候語
        random_greeting = random.choice(greetings)

        # 將資料寫入檔案
        with open(filename, "a", encoding="utf-8") as file:
            file.write(log_entry_to_file)

        # 顯示包含標籤的確認訊息
        print(display_message)
        # 顯示隨機問候語
        print(random_greeting)
        print("-" * 20)

    print("程式已結束。所有紀錄已儲存至 log.txt。")

# 執行函式
if __name__ == "__main__":
    record_data()

歡迎使用紀錄程式。
請按照提示輸入，輸入 'q' 退出。
--------------------
請輸入日期 (例如: 2025-09-11 或 'today'): today
請輸入品項: bread
請輸入時間 (例如: 14:30 或 'now'): 14:30
請輸入金額: 50
成功新增紀錄: 日期: 2025-09-11, 品項: bread, 時間: 14:30, 金額: 50.0
祝您有愉快的一天！
--------------------
請輸入日期 (例如: 2025-09-11 或 'today'): today
請輸入品項: umbrella
請輸入時間 (例如: 14:30 或 'now'): 15:10
請輸入金額: 199
成功新增紀錄: 日期: 2025-09-11, 品項: umbrella, 時間: 15:10, 金額: 199.0
祝您有愉快的一天！
--------------------
請輸入日期 (例如: 2025-09-11 或 'today'): q
程式已結束。所有紀錄已儲存至 log.txt。


In [43]:
import datetime
import random
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def record_data(sheet):
    greetings = [
        "別花太多錢喔！",
        "祝您有愉快的一天！",
        "今天也要努力賺錢！",
        "繼續保持好習慣！",
        "每一筆花費都值得！"
    ]

    print("歡迎使用紀錄程式。")
    print("請按照提示輸入，輸入 'q' 退出。")
    print("-" * 20)

    # 檢查標題列
    current_rows = sheet.get_all_values()
    if not current_rows or current_rows[0] != ["日期", "品項", "時間", "金額"]:
        sheet.append_row(["日期", "品項", "時間", "金額"])

    while True:
        # 日期
        date_input = input("請輸入日期 (例如: 2025-09-11 或 'today'): ")
        if date_input.lower() == 'q':
            break
        if date_input.lower() == 'today':
            date_to_record = datetime.date.today().strftime('%Y-%m-%d')
        else:
            try:
                datetime.datetime.strptime(date_input, '%Y-%m-%d')
                date_to_record = date_input
            except ValueError:
                print("日期格式不正確，請使用 YYYY-MM-DD 格式。")
                continue

        # 品項
        item_input = input("請輸入品項: ")
        if item_input.lower() == 'q':
            break

        # 時間
        time_input = input("請輸入時間 (例如: 14:30 或 'now'): ")
        if time_input.lower() == 'q':
            break
        if time_input.lower() == 'now':
            time_to_record = datetime.datetime.now().strftime('%H:%M')
        else:
            try:
                datetime.datetime.strptime(time_input, '%H:%M')
                time_to_record = time_input
            except ValueError:
                print("時間格式不正確，請使用 HH:MM 格式。")
                continue

        # 金額
        amount_input = input("請輸入金額: ")
        if amount_input.lower() == 'q':
            break
        try:
            amount_to_record = float(amount_input)
        except ValueError:
            print("金額格式不正確，請輸入數字。")
            continue

        # 寫入 Google Sheet
        row_data = [date_to_record, item_input, time_to_record, amount_to_record]
        try:
            sheet.append_row(row_data)
        except Exception as e:
            print("寫入 Google 試算表時發生錯誤:", e)
            continue

        # 顯示訊息
        print(f"成功新增紀錄: 日期: {row_data[0]}, 品項: {row_data[1]}, 時間: {row_data[2]}, 金額: {row_data[3]}")
        print(random.choice(greetings))
        print("-" * 20)

    print("程式已結束。所有紀錄已儲存至 Google 試算表。")


if __name__ == "__main__":
    # Google Sheets 授權

    gc = gspread.authorize(creds)

    # 打開試算表（可改用 open 或 open_by_url）
    sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1X0uCEvhBCrJ-M8_nWAnGi0bcacp_Nzt5-V_z5rK8rPc/edit?gid=0#gid=0").sheet1

    # 執行紀錄功能
    record_data(sheet)


歡迎使用紀錄程式。
請按照提示輸入，輸入 'q' 退出。
--------------------
請輸入日期 (例如: 2025-09-11 或 'today'): today
請輸入品項: 麵包
請輸入時間 (例如: 14:30 或 'now'): 14:30
請輸入金額: 50
成功新增紀錄: 日期: 2025-09-11, 品項: 麵包, 時間: 14:30, 金額: 50.0
別花太多錢喔！
--------------------
請輸入日期 (例如: 2025-09-11 或 'today'): q
程式已結束。所有紀錄已儲存至 Google 試算表。
